In [0]:
import os
import re
import datetime
import numpy as np
from PIL import Image
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
def prepae_data():
    x, t = [], []

    categorized_dir_paths = glob('/content/drive/My Drive/kikagaku/novelapp/data/images/categorized-2/*')

    for dir_path in categorized_dir_paths:
        category_id = os.path.basename(dir_path)
        image_paths = glob(os.path.join(dir_path, '*'))
        print(datetime.datetime.now().isoformat(), 'Category', category_id, ':', len(image_paths))

        for i, p in enumerate(image_paths):
            book_id = re.sub(r'(_.*$)', '', os.path.basename(p))

            if i % 30 == 0:
                print(datetime.datetime.now().isoformat(), 'Image:', i, book_id, category_id)

            try:
                img = Image.open(p)
                img_resize = img.resize((229, 229))
                img_np = np.array(img_resize) / 255.0
                img_reshape = img_np.reshape(1, 229, 229, 3)
                #print(datetime.datetime.now().isoformat(), img_np.shape, img_reshape.shape)

                x.append(img_np)
                t.append(category_id)

            except Exception as e:
                print(datetime.datetime.now().isoformat(), 'Error:', e)

    return x, t

In [3]:
x, t = prepae_data()

2020-06-05T09:16:29.548943 Category 0 : 117
2020-06-05T09:16:29.549688 Image: 0 1015 0
2020-06-05T09:16:49.974705 Image: 30 1228 0
2020-06-05T09:17:10.239447 Image: 60 1106 0
2020-06-05T09:17:29.470165 Image: 90 1129 0
2020-06-05T09:17:47.186211 Category 1 : 119
2020-06-05T09:17:47.186426 Image: 0 3636 1
2020-06-05T09:18:06.556183 Image: 30 3722 1
2020-06-05T09:18:25.942979 Image: 60 3832 1
2020-06-05T09:18:43.713511 Image: 90 3777 1
2020-06-05T09:19:00.977256 Category 2 : 129
2020-06-05T09:19:00.977878 Image: 0 3601 2
2020-06-05T09:19:19.731418 Image: 30 3884 2
2020-06-05T09:19:38.482287 Image: 60 3778 2
2020-06-05T09:19:57.171130 Image: 90 3853 2
2020-06-05T09:20:16.032931 Image: 120 1502 2
2020-06-05T09:20:21.551193 Category 3 : 111
2020-06-05T09:20:21.551809 Image: 0 3312 3
2020-06-05T09:20:40.207119 Image: 30 2350 3
2020-06-05T09:20:59.057234 Image: 60 3010 3
2020-06-05T09:21:17.364931 Image: 90 3105 3


In [4]:
x = np.array(x).astype('f')
t = np.array(t).astype('f')
print(x.shape, t.shape)

(476, 229, 229, 3) (476,)


In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, t_train, t_val = train_test_split(x, t, test_size=0.3, random_state=0)

In [7]:
print(x_train.shape, x_val.shape)

(333, 229, 229, 3) (143, 229, 229, 3)


In [0]:
import os
import random

def reset_seed(seed=0):

    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [0]:
reset_seed(0)

In [10]:
category_count = len(np.unique(t))
print(np.unique(t))
print(category_count)

[0. 1. 2. 3.]
4


In [0]:
from tensorflow.keras import models, layers
from tensorflow.keras.applications import Xception
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=180,     # randomly rotate images in the range
    zoom_range=0.1,         # randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally
    height_shift_range=0.1, # randomly shift images vertically
    horizontal_flip=True,   # randomly flip images horizontally
    vertical_flip=True      # randomly flip images vertically
)

datagen.fit(x_train)

In [0]:
def build_model(units, dropout, lr):
  model_fine = Xception(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

  for layer in model_fine.layers[:100]:
      layer.trainable = False

  model = models.Sequential()
  model.add(model_fine)

  model.add(layers.Flatten())

  model.add(layers.BatchNormalization())
  model.add(layers.Dense(units, activation='relu'))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(category_count, activation='softmax'))

  #optimizer = keras.optimizers.Adagrad(lr=lr)
  #optimizer = keras.optimizers.Adam(lr=lr)
  optimizer = keras.optimizers.SGD(lr=lr)

  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model

In [14]:
print(datetime.datetime.now().isoformat(), 'Start')

units = [10, 20, 40, 60, 120]
dropout = [0.1, 0.2, 0.3, 0.4, 0.5]
lr = [0.01, 0.005, 0.002, 0.001, 0.0008]
batch_size = [10, 20, 30, 60, 100]

param_dict = dict(units=units, 
                  dropout=dropout,
                  lr=lr, 
                  batch_size=batch_size)

model = KerasClassifier(build_fn = build_model, verbose=1)

rand = RandomizedSearchCV(estimator=model, param_distributions=param_dict, n_iter=10)

rand_result = rand.fit(datagen.flow(x_train, t_train, batch_size=32))

#history = model.fit_generator(
#    datagen.flow(x_train, t_train, batch_size=32),
#    epochs=100,
#    verbose=2,
#    validation_data=(x_val, t_val),
#    #steps_per_epoch=x_train.shape[0]
#)

print(datetime.datetime.now().isoformat(), 'End')

2020-06-05T09:24:38.582897 Start


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: fit() missing 1 required positional argument: 'y'

  FitFailedWarning)


TypeError: ignored